In [15]:
import os 
import numpy as np
import pickle

class Relation_Config(object):
    def __init__(self):
        self.d_g = 64
        self.d_k = 64
        self.Nr = 16

num_samples_on_each_model = 5000
n_object_per_image_in_training = 8

NYU40CLASSES = ['void',
                'wall', 'floor', 'cabinet', 'bed', 'chair',
                'sofa', 'table', 'door', 'window', 'bookshelf',
                'picture', 'counter', 'blinds', 'desk', 'shelves',
                'curtain', 'dresser', 'pillow', 'mirror', 'floor_mat',
                'clothes', 'ceiling', 'books', 'refridgerator', 'television',
                'paper', 'towel', 'shower_curtain', 'box', 'whiteboard',
                'person', 'night_stand', 'toilet', 'sink', 'lamp',
                'bathtub', 'bag', 'otherstructure', 'otherfurniture', 'otherprop']

NYU37_TO_PIX3D_CLS_MAPPING = {0:0, 1:0, 2:0, 3:8, 4:1, 5:3, 6:5, 7:6, 8:8, 9:2, 10:2, 11:0, 12:0, 13:2, 14:4,
                              15:2, 16:2, 17:8, 18:0, 19:0, 20:0, 21:0, 22:0, 23:0, 24:8, 25:8, 26:0, 27:0, 28:0,
                              29:8, 30:8, 31:0, 32:8, 33:0, 34:0, 35:0, 36:0, 37:8}

RECON_3D_CLS = [3,4,5,6,7,8,10,14,15,17,24,25,29,30,32]

number_pnts_on_template = 2562

pix3d_n_classes = 9

cls_reg_ratio = 10
obj_cam_ratio = 1

class Config(object):
    def __init__(self, dataset):
        # data path configuration
        self.dataset = dataset
        
        if self.dataset == 'sunrgbd':
            self.metadata_path = './data/sunrgbd'
            self.trian_test_data_path = os.path.join(self.metadata_path, 'sunrgb_train_test_data')
            self.__size_avg_path = os.path.join(self.metadata_path, 'preprocessed/size_avg_category.pkl')
            self.__layout_avg_file = os.path.join(self.metadata_path, 'preprocessed/layout_avg_file.pkl')
            self.bins = self.__initate_bins()
            self.evaluation_path = './evaluation/sunrgbd'
            # check if trian_test_data_path exist
            if not os.path.exists(self.trian_test_data_path):
                os.mkdir(self.trian_test_data_path)
        
    def __initate_bins(self):
        bin = {}

        if self.dataset == 'sunrgbd':
            #use priors for layout  regression
            if os.path.exists(self.__layout_avg_file):
                with open(self.__layout_avg_file, 'rb') as file:
                    layout_avg_dict = pickle.load(file)
            else:
                raise IOError('No layout average file in %s. Please check.' % (self.__layout_avg_file))

            bin['layout_centroid_avg'] = layout_avg_dict['layout_centroid_avg']
            bin['layout_coeffs_avg'] = layout_avg_dict['layout_coeffs_avg']

            # layout orientation bin
            NUM_LAYOUT_ORI_BIN = 2
            ORI_LAYOUT_BIN_WIDTH = np.pi / 4
            bin['layout_ori_bin'] = [[np.pi / 4 + i * ORI_LAYOUT_BIN_WIDTH, np.pi / 4 + (i + 1) * ORI_LAYOUT_BIN_WIDTH] 
                                     for i in range(NUM_LAYOUT_ORI_BIN)]

            # camera bin
            PITCH_NUMBER_BINS = 2
            PITCH_WIDTH = 40 * np.pi / 180
            ROLL_NUMBER_BINS = 2
            ROLL_WIDTH = 20 * np.pi / 180

            bin['pitch_bin'] = [[-60.0 * np.pi / 180 + i * PITCH_WIDTH, -60.0 * np.pi / 180 + (i + 1) * PITCH_WIDTH] for
                            i in range(PITCH_NUMBER_BINS)]
            bin['roll_bin'] = [[-20.0 * np.pi / 180 + i * ROLL_WIDTH, -20.0 * np.pi / 180 + (i + 1) * ROLL_WIDTH] for i in
                           range(ROLL_NUMBER_BINS)]

            # bbox orin, size and centroid bin
            #orin bin
            NUM_ORI_BIN = 6
            # 60 degrees width for each bin.
            ORI_BIN_WIDTH = float(2 * np.pi / NUM_ORI_BIN)
            # orientation bin ranges from -np.pi to np.pi.
            bin['ori_bin'] = [[(i - NUM_ORI_BIN / 2) * ORI_BIN_WIDTH, (i - NUM_ORI_BIN / 2 + 1) * ORI_BIN_WIDTH] 
                              for i in range(NUM_ORI_BIN)]
            #check if size_avg_path exist
            if os.path.exists(self.__size_avg_path):
                with open(self.__size_avg_path, 'rb') as file:
                    avg_size = pickle.load(file)
            else:
                raise IOError('No object average size file in %s. Please check.' % (self.__size_avg_path))

            bin['avg_size'] = np.vstack([avg_size[key] for key in range(len(avg_size))])

            # for each object bbox, the distance between camera and object centroid will be estimated.
            NUM_DEPTH_BIN = 6
            DEPTH_WIDTH = 1.0
            bin['centroid_bin'] = [[i * DEPTH_WIDTH, (i+1) * DEPTH_WIDTH] for i in range(NUM_DEPTH_BIN)]
        else:
            raise NameError('Please specify a correct dataset name.')

        return bin

In [16]:
config = Config('sunrgbd')

In [18]:
config.bins

{'layout_centroid_avg': [1.616122489081186,
  1.3317608169123132,
  -0.04355525682072995],
 'layout_coeffs_avg': [2.417306821511964,
  2.622592360862724,
  1.7938134221434006],
 'layout_ori_bin': [[0.7853981633974483, 1.5707963267948966],
  [1.5707963267948966, 2.356194490192345]],
 'pitch_bin': [[-1.0471975511965976, -0.34906585039886584],
  [-0.34906585039886584, 0.34906585039886595]],
 'roll_bin': [[-0.3490658503988659, 0.0], [0.0, 0.3490658503988659]],
 'ori_bin': [[-3.141592653589793, -2.0943951023931953],
  [-2.0943951023931953, -1.0471975511965976],
  [-1.0471975511965976, 0.0],
  [0.0, 1.0471975511965976],
  [1.0471975511965976, 2.0943951023931953],
  [2.0943951023931953, 3.141592653589793]],
 'avg_size': array([[0.3384985 , 0.28602166, 0.22452457],
        [0.14408408, 0.165625  , 0.04398023],
        [0.40244022, 0.36310268, 0.28916155],
        [0.68561647, 0.60170927, 0.28520731],
        [0.7692062 , 0.5118861 , 1.00517711],
        [0.27750872, 0.41992745, 0.29389724],
  